In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib
import matplotlib.pyplot as plt
import math

# Load your dataset here
data = pd.read_csv('training.csv')
negativewords = pd.read_csv('negative-words.csv')
negativewords = negativewords.rename(columns={'2-faced': 'neg'})
negativewords['val'] = [-1 for _ in range(len(negativewords))]
positivewords = pd.read_csv('positive-words.csv')
positivewords = positivewords.rename(columns={'a+': 'pos'})
positivewords['val'] = [1 for _ in range(len(positivewords))]

In [5]:
def parsingwords(s):
  words = s.split(' ')
  val = []
  for n in list(negativewords['neg']):
    if s.find(str(n)) != -1:
      val.append(-1)
  for p in positivewords['pos']:
    if s.find(p) != -1:
      val.append(1)
  if len(val) != 0:
    return sum(val)/len(val)
  return 0

vals = []
for row in range(len(data)) :
  #print(data['text'][row], data[])
  vals.append(parsingwords(data['text'][row]))
  #print(vals[row], '\n\n')
data['vals'] = [parsingwords(data['text'][row]) for row in range(len(data))]
data.head(25)
#print([parsingwords(data['text'][row]) for row in range(2)])
#print([data['text'][row] for row in range(2)])
# 0 = sad
# 1 = joy
# 2 = love
# 3 = anger
# 4 = fear
# 5 = surprise

,text,label,vals
0,i didnt feel humiliated,0,-1.000000
1,i can go from feeling so hopeless to so damned...,0,-0.500000
2,im grabbing a minute to post i feel greedy wrong,3,-1.000000
3,i am ever feeling nostalgic about the fireplac...,2,0.000000
4,i am feeling grouchy,3,-1.000000
5,ive been feeling a little burdened lately wasn...,0,-1.000000
6,ive been taking or milligrams or times recomme...,5,0.400000
7,i feel as confused about life as a teenager or...,4,-1.000000
8,i have been with petronas for years i feel tha...,1,0.333333
9,i feel romantic too,2,1.000000
